In [ ]:
!pip install --q git+https://github.com/m-bain/whisperx.git

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.6/208.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.6/630.6 kB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import whisperx
import gc

In [ ]:
device = "cuda"
batch_size = 16 # reduce if low on GPU mem
compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)

In [ ]:
audio_file = "Neural.WAV"


In [ ]:
# 1. Transcribe with original whisper (batched)
model = whisperx.load_model("large-v2", device, compute_type=compute_type)

config.json:   0%|          | 0.00/2.80k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

No language specified, language will be first be detected for each audio file (increases inference time).


100%|█████████████████████████████████████| 16.9M/16.9M [00:01<00:00, 11.1MiB/s]
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.1.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.0. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.1.0+cu118. Bad things might happen unless you revert torch to 1.x.


In [ ]:
audio = whisperx.load_audio(audio_file)
result = model.transcribe(audio, batch_size=batch_size)
print(result["segments"]) # before alignment

Detected language: en (1.00) in first 30s of audio...
[{'text': ' We have a new leader on the 7 billion parameter models on the Hugging Face leaderboard. And this one is from a surprising company. So Intel released their first LLM which they are calling NeuralChat 7B. Now this was fine-tuned on their new hardware platform called Habana Guardi2.', 'start': 1.271, 'end': 23.78}]


In [ ]:
result

{'segments': [{'text': ' We have a new leader on the 7 billion parameter models on the Hugging Face leaderboard. And this one is from a surprising company. So Intel released their first LLM which they are calling NeuralChat 7B. Now this was fine-tuned on their new hardware platform called Habana Guardi2.',
   'start': 1.271,
   'end': 23.78}],
 'language': 'en'}

### Word Level Transcription

In [ ]:
# 2. Align whisper output
model_a, metadata = whisperx.load_align_model(language_code=result["language"],
                                              device=device)

result = whisperx.align(result["segments"], model_a,
                        metadata,
                        audio,
                        device,
                        return_char_alignments=False)

print(result["segments"]) # after alignment

Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_base_ls960_asr_ls960.pth" to /root/.cache/torch/hub/checkpoints/wav2vec2_fairseq_base_ls960_asr_ls960.pth
100%|██████████| 360M/360M [00:01<00:00, 240MB/s]


[{'start': 1.331, 'end': 5.953, 'text': ' We have a new leader on the 7 billion parameter models on the Hugging Face leaderboard.', 'words': [{'word': 'We', 'start': 1.331, 'end': 1.471, 'score': 0.827}, {'word': 'have', 'start': 1.491, 'end': 1.651, 'score': 0.89}, {'word': 'a', 'start': 1.691, 'end': 1.711, 'score': 0.996}, {'word': 'new', 'start': 1.771, 'end': 1.891, 'score': 0.999}, {'word': 'leader', 'start': 1.951, 'end': 2.231, 'score': 0.925}, {'word': 'on', 'start': 2.371, 'end': 2.451, 'score': 0.943}, {'word': 'the', 'start': 2.511, 'end': 2.672, 'score': 0.94}, {'word': '7'}, {'word': 'billion', 'start': 3.352, 'end': 3.692, 'score': 0.851}, {'word': 'parameter', 'start': 3.732, 'end': 4.152, 'score': 0.817}, {'word': 'models', 'start': 4.192, 'end': 4.472, 'score': 0.703}, {'word': 'on', 'start': 4.612, 'end': 4.692, 'score': 0.855}, {'word': 'the', 'start': 4.712, 'end': 4.812, 'score': 0.75}, {'word': 'Hugging', 'start': 4.832, 'end': 5.093, 'score': 0.914}, {'word': 'F

In [ ]:
result["segments"]

[{'start': 1.331,
  'end': 5.953,
  'text': ' We have a new leader on the 7 billion parameter models on the Hugging Face leaderboard.',
  'words': [{'word': 'We', 'start': 1.331, 'end': 1.471, 'score': 0.827},
   {'word': 'have', 'start': 1.491, 'end': 1.651, 'score': 0.89},
   {'word': 'a', 'start': 1.691, 'end': 1.711, 'score': 0.996},
   {'word': 'new', 'start': 1.771, 'end': 1.891, 'score': 0.999},
   {'word': 'leader', 'start': 1.951, 'end': 2.231, 'score': 0.925},
   {'word': 'on', 'start': 2.371, 'end': 2.451, 'score': 0.943},
   {'word': 'the', 'start': 2.511, 'end': 2.672, 'score': 0.94},
   {'word': '7'},
   {'word': 'billion', 'start': 3.352, 'end': 3.692, 'score': 0.851},
   {'word': 'parameter', 'start': 3.732, 'end': 4.152, 'score': 0.817},
   {'word': 'models', 'start': 4.192, 'end': 4.472, 'score': 0.703},
   {'word': 'on', 'start': 4.612, 'end': 4.692, 'score': 0.855},
   {'word': 'the', 'start': 4.712, 'end': 4.812, 'score': 0.75},
   {'word': 'Hugging', 'start': 4.83